In [1]:
import yfinance as yf
from proxy import enable
from lorentzian import lorentzian_distance, lorentzian_k_nearest_neighbor, Diction, KNode, test_data_by_df, \
    prediction2node
from features import n_rsi, n_adx, n_cci, n_wt, HLC3, FeaturesX, FeaturesDefinition
import ta.wrapper

enable()

In [2]:
apple = yf.Ticker("AAPL")

In [3]:
his = apple.history(period="max", interval="1d")

In [31]:
his['SMA20'] = ta.wrapper.SMAIndicator(his['Close'], window=200).sma_indicator()
his['EMA20'] = ta.wrapper.EMAIndicator(his['Close'], window=200).ema_indicator()

In [32]:
his['SMA20_-1d'] = his["SMA20"] - his["SMA20"].shift(1)
his['EMA20_-1d'] = his["EMA20"] - his["EMA20"].shift(1)

In [33]:
features_definition = FeaturesDefinition.default(high=his['High'], close=his['Close'], open=his['Open'], low=his['Low'])
his = features_definition.calculate(his)

In [39]:
test_data = test_data_by_df(his, features_definition, future_count=8)

In [40]:
start_index = int(len(test_data) * 0.8)
train = test_data[:start_index]
test = test_data[start_index:]

In [41]:
def buy_signal(the_current, the_test):
    return the_test.info['SMA20_-1d'] > 0 and the_test.info[
        'EMA20_-1d'] > 0 and the_current.diction == Diction.LONG


def sell_signal(the_current, the_test):
    return the_test.info['SMA20_-1d'] < 0 and the_test.info[
        'EMA20_-1d'] < 0 and the_current.diction == Diction.SHORT

In [42]:
win_num = 0
buy_num = 0
sell_num = 0
pos_index = 1
total = 0
for k_node in test:
    current_node = prediction2node(k_node.info, train, features_definition,k=8)
    if buy_signal(current_node, k_node):
        buy_num = buy_num + 1
        if current_node.diction == k_node.diction:
            win_num = win_num + 1
    elif sell_signal(current_node, k_node):
        sell_num = sell_num + 1
        if current_node.diction == k_node.diction:
            win_num = win_num + 1        

    total = buy_num + sell_num
    if total > 0:
        print(
            f'progress: {pos_index}/{len(test)} | buy: {buy_num} | sell: {sell_num} | Total: {total} | Win: {win_num} | Win Rate: {win_num / total * 100:.2f}%',
            end='\r', flush=True)
    pos_index = pos_index + 1

print('\n\ntest end.\n')
print(
    f'Win Rate: {win_num / total * 100:.2f}% | Win: {win_num} | Total: {total} | buy: {buy_num} | sell: {sell_num}')

progress: 2113/2113 | buy: 742 | sell: 129 | Total: 871 | Win: 514 | Win Rate: 59.01%

test end.

Win Rate: 59.01% | Win: 514 | Total: 2113 | buy: 742 | sell: 129


In [ ]:
his['SMA20_-1d']

In [ ]:
node = prediction2node(his.iloc[-1], test_data, features_definition)